# Scrape news and Analyse sentiments
This notebook shows an example of scraping news articles linked to specific traded companies and utilizing our predeployed sentiment analysis model server to predict the sentiment of the author towards said companies.

In [1]:
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio 

## Environment

In [2]:
%nuclio env -c V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env -c V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env -c V3IO_API=${V3IO_API}

In [3]:
%%nuclio cmd -c
pip install beautifulsoup4
pip install pandas
pip install v3io_frames

In [4]:
%nuclio config kind = "nuclio"
%nuclio config spec.build.baseImage = "python:3.6-jessie"

%nuclio: setting kind to 'nuclio'
%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'


## Function

In [5]:
# nuclio: start-code

In [6]:
from bs4 import BeautifulSoup as bs
from urllib.request import Request, urlopen
import requests
import pandas as pd
import v3io_frames as v3f
from unicodedata import normalize
from datetime import datetime
import re
import os
import json

In [7]:
def get_stock_news_page(stock_string):
    request = Request('https://www.investing.com/equities/' + stock_string + '-news', headers={"User-Agent": "Mozilla/5.0"})
    content = urlopen(request).read()
    return bs(content, 'html.parser')

def get_internal_article_links(page):
    news = page.find_all('div', attrs={'class': 'mediumTitle1'})[1]
    articles = news.find_all('article', attrs={'class': 'js-article-item articleItem'})
    return ['https://www.investing.com' + a.find('a').attrs['href'] for a in articles]

def get_article_page(article_link):
    request = Request(article_link, headers={"User-Agent": "Mozilla/5.0"})
    content = urlopen(request).read()
    return bs(content, 'html.parser')

def clean_paragraph(paragraph):
    paragraph = re.sub(r'\(http\S+', '', paragraph)
    paragraph = re.sub(r'\([A-Z]+:[A-Z]+\)', '', paragraph)
    paragraph = re.sub(r'[\n\t\s\']', ' ', paragraph)
    return normalize('NFKD', paragraph)    

def extract_text(article_page):
    text_tag = article_page.find('div', attrs={'class': 'WYSIWYG articlePage'})
    paragraphs = text_tag.find_all('p')
    text = '\n'.join([clean_paragraph(p.get_text()) for p in paragraphs[:-1]])
    return text

def get_publish_time(article_page):
    details = article_page.find('meta', attrs={'itemprop': 'dateModified'})
    publish_date = details.get_attribute_list('content')[0]
    return str(datetime.strptime(publish_date, '%Y-%m-%d %H:%M:%S'))

def get_score(paragraph_scores):
    return sum([score - 1 for score in paragraph_scores]) / len(paragraph_scores)  

def get_article_scores(context, articles, endpoint):
    scores = [] 
    for i, article in enumerate(articles):
        context.logger.info(f'getting score for article {i + 1}\\{len(articles)}')
        event_data = {'instances': article.split('\n')}
        resp = requests.put(endpoint+'/bert_classifier_v1/predict', json=json.dumps(event_data))
        scores.append(get_score(json.loads(resp.text)))
    return scores

In [8]:
def init_context(context):
    # Setup V3IO Client
    client = v3f.Client('framesd:8081',container=os.getenv('V3IO_CONTAINER', 'bigdata'))
    setattr(context, 'v3c', client)
    
    # Create stocks stream
    setattr(context, 'stocks_stream', os.getenv('STOCKS_STREAM', 'stocks/stocks_stream'))
    context.v3c.create(backend='stream', table=context.stocks_stream, if_exists=1)
    
    # Create TSDB table
    setattr(context, 'stocks_tsdb', os.getenv('STOCKS_TSDB_TABLE', 'stocks/stocks_tsdb'))
    context.v3c.create(backend='tsdb', table=context.stocks_tsdb, rate='1/s', if_exists=1)
    
    # Supply the endpoint provided at the end of execution of 00-deploy-sentiment-model.ipynb.
    setattr(context, 'sentiment_model_endpoint', os.getenv('SENTIMENT_MODEL_ENDPOINT', 'http://nuclio-stocks-sentiment-analysis-server:8080'))

    sym_to_url={'GOOGL': 'google-inc', 'MSFT': 'microsoft-corp', 'AMZN': 'amazon-com-inc', 'AAPL': 'apple-computer-inc'}
    setattr(context, 'sym_to_url', sym_to_url)
    setattr(context, 'stocks_kv', os.getenv('STOCKS_KV', 'stocks/stocks_kv'))

In [9]:
def handler(context, handler):
    
    syms = []
    contents = []
    links = []
    times = []
    sentiments = []
    
    for sym, url_string in context.sym_to_url.items():
        context.logger.info(f'Getting news about {sym}')
        news_page = get_stock_news_page(url_string)
        article_links = get_internal_article_links(news_page)
        article_pages = [get_article_page(link) for link in article_links]
        articles = [extract_text(article_page) for article_page in article_pages]
        curr_sentiments = get_article_scores(context, articles, context.sentiment_model_endpoint)
        curr_times = [get_publish_time(article_page) for article_page in article_pages]
        
        sentiments += curr_sentiments
        times += curr_times
        for article, link, sentiment, time in zip(articles, article_links, curr_sentiments, curr_times):
            record = {
                'content': article,
                'time': time,
                'symbol': sym,
                'link': link,
                'sentiment': sentiment
            }
            context.v3c.execute('stream', context.stocks_stream, 'put', args={'data': json.dumps(record)})
            
            
            syms.append(sym)
            contents.append(article)
            links.append(link)
        context.v3c.execute('kv', context.stocks_kv, command='update', args={'key': sym,
                                                                             'expression': f'SET sentiment={sentiments[-1]}'})
                  
    if len(sentiments)>0:
        df = pd.DataFrame.from_dict({'sentiment': sentiments,
                                     'time': times,
                                     'symbol': syms})
        df = df.set_index(['time', 'symbol'])
        df.index = df.index.set_levels([pd.to_datetime(df.index.levels[0]), df.index.levels[1]])
        df = df.sort_index(level=0, axis=0)
        context.v3c.write(backend='tsdb', table=context.stocks_tsdb, dfs=df)

In [10]:
# nuclio: end-code

## Test locally

In [11]:
init_context(context)

In [12]:
from nuclio import Event
event = Event()

In [13]:
handler(context, event)

Python> 2020-10-18 07:37:20,858 [info] Getting news about GOOGL
Python> 2020-10-18 07:37:32,099 [info] getting score for article 1\6
Python> 2020-10-18 07:37:33,830 [info] getting score for article 2\6
Python> 2020-10-18 07:37:35,535 [info] getting score for article 3\6
Python> 2020-10-18 07:37:36,975 [info] getting score for article 4\6
Python> 2020-10-18 07:37:37,670 [info] getting score for article 5\6
Python> 2020-10-18 07:37:38,311 [info] getting score for article 6\6
Python> 2020-10-18 07:37:39,058 [info] Getting news about MSFT
Python> 2020-10-18 07:37:55,395 [info] getting score for article 1\9
Python> 2020-10-18 07:37:56,646 [info] getting score for article 2\9
Python> 2020-10-18 07:37:58,076 [info] getting score for article 3\9
Python> 2020-10-18 07:37:59,197 [info] getting score for article 4\9
Python> 2020-10-18 07:38:00,132 [info] getting score for article 5\9
Python> 2020-10-18 07:38:01,556 [info] getting score for article 6\9
Python> 2020-10-18 07:38:02,083 [info] gettin

## Deploy to cluster

In [14]:
from mlrun import code_to_function

# Export bare function
fn = code_to_function('read-news',
                      handler='handler')
fn.export('02-read-news.yaml')

# Set parameters for current deployment
fn.add_trigger('cron', nuclio.triggers.CronTrigger('300s'))
fn.set_envs({'V3IO_CONTAINER': 'bigdata',
             'STOCKS_STREAM': 'stocks/stocks_stream',
             'STOCKS_TSDB_TABLE': 'stocks/stocks_tsdb',
             'SENTIMENT_MODEL_ENDPOINT': 'http://nuclio-stocks-sentiment-analysis-serving:8080'})
fn.spec.max_replicas = 1

> 2020-10-14 07:00:08,593 [info] function spec saved to path: 02-read-news.yaml


In [15]:
fn.deploy(project='stocks')

> 2020-10-14 07:00:08,600 [info] deploy started
[nuclio] 2020-10-14 07:01:08,879 (info) Build complete
[nuclio] 2020-10-14 07:01:17,040 (info) Function deploy complete
[nuclio] 2020-10-14 07:01:17,048 done updating stocks-read-news, function address: 192.168.224.209:30776
> 2020-10-14 07:01:17,057 [warning] warning!, server (0.5.1) and client (0.5.2) ver dont match


'http://192.168.224.209:30776'